# 神經網路實做

## 拿取加權指數資料

In [22]:
from finlab.data import Data

data = Data()
twii = data.get("發行量加權股價指數")

twii = twii['台股指數'].resample("15T").first().dropna()
twii.head()

date
2006-01-02 09:00:00    6548.34
2006-01-02 09:15:00    6478.09
2006-01-02 09:30:00    6474.88
2006-01-02 09:45:00    6471.12
2006-01-02 10:00:00    6480.50
Name: 台股指數, dtype: float64

## 製作features

In [23]:
import talib
import numpy as np
import pandas as pd

sma = talib.SMA(twii, timeperiod=120)
wma = talib.WMA(twii, timeperiod=120)
mom = talib.MOM(twii, timeperiod=120)
k, d = talib.STOCH  (twii, twii, twii, fastk_period=120, slowk_period=60, slowd_period=60)
k2, d2 = talib.STOCH(twii, twii, twii, fastk_period=240, slowk_period=120, slowd_period=120)
k3, d3 = talib.STOCH(twii, twii, twii, fastk_period=360, slowk_period=180, slowd_period=180)
k4, d4 = talib.STOCH(twii, twii, twii, fastk_period=480, slowk_period=240, slowd_period=240)
k5, d5 = talib.STOCH(twii, twii, twii, fastk_period=640, slowk_period=320, slowd_period=320)
k6, d6 = talib.STOCH(twii, twii, twii, fastk_period=720, slowk_period=360, slowd_period=360)
k7, d7 = talib.STOCH(twii, twii, twii, fastk_period=840, slowk_period=420, slowd_period=420)
k8, d8 = talib.STOCH(twii, twii, twii, fastk_period=960, slowk_period=480, slowd_period=480)

rsi = talib.RSI (twii, timeperiod=120)
rsi2 = talib.RSI(twii, timeperiod=240)
rsi3 = talib.RSI(twii, timeperiod=480)
rsi4 = talib.RSI(twii, timeperiod=640)
rsi5 = talib.RSI(twii, timeperiod=720)
rsi6 = talib.RSI(twii, timeperiod=840)

macd1, macd2, macd3 = talib.MACD(twii, fastperiod=120, slowperiod=60, signalperiod=60)
willr = talib.WILLR(twii, twii, twii, timeperiod=120)
cci = talib.CCI(twii, twii, twii, timeperiod=120)

dataset = pd.DataFrame({
    'RSIb': rsi / 50,
    'RSIb2': rsi2 / 50,
    'RSIb3': rsi3 / 50,
    'RSIb4': rsi4 / 50,
    'RSIb5': rsi5 / 50,
    'RSIb6': rsi6 / 50,
    'MOMb': mom - 0,
    'KDb': k - d,
    'KDb2': k2 - d2,
    'KDb3': k3 - d3,
    'KDb4': k4 - d4,
    'KDb5': k5 - d5,
    'KDb6': k6 - d6,
    'KDb7': k7 - d7,
    'KDb8': k8 - d8,
    
    'a5':   (twii.rolling(5).mean()   / twii),
    'a10':  (twii.rolling(10).mean()  / twii),
    'a20':  (twii.rolling(20).mean()  / twii),
    'a40':  (twii.rolling(40).mean()  / twii),
    'a80':  (twii.rolling(80).mean()  / twii),
    'a160': (twii.rolling(160).mean() / twii),
    'a320': (twii.rolling(320).mean() / twii),
    'a640': (twii.rolling(640).mean() / twii),
    'a720': (twii.rolling(720).mean() / twii),
    'a840': (twii.rolling(840).mean() / twii),
    'a960': (twii.rolling(960).mean() / twii),
    'a1024':(twii.rolling(1024).mean() / twii),
    'b1': twii/twii.shift(50),
    'b2': twii/twii.shift(100),
    'b3': twii/twii.shift(150),
    'b4': twii/twii.shift(200),
    'b5': twii/twii.shift(250),
    'b6': twii/twii.shift(300),
    'b7': twii/twii.shift(350),
    'LINEARREG_SLOPE0': talib.LINEARREG_SLOPE(twii, 60),
    'LINEARREG_SLOPE1': talib.LINEARREG_SLOPE(twii, 120),

    'ADXR0': talib.ADXR(twii, twii, twii, 60),
    'ADXR1': talib.ADXR(twii, twii, twii, 120),
    'ADXR2': talib.ADXR(twii, twii, twii, 240),
    'ADXR3': talib.ADXR(twii, twii, twii, 360),
    'ADXR4': talib.ADXR(twii, twii, twii, 480),
    'ADXR5': talib.ADXR(twii, twii, twii, 640),

    'return': twii.shift(-10) / twii,
})

feature_names = list(dataset.columns[:-1])

## 簡單處理一下

In [24]:
print("before dropping NaN", dataset.shape)
dataset = dataset.dropna()
print("after dropping NaN", dataset.shape)


before dropping NaN (79078, 43)
after dropping NaN (77150, 43)


In [25]:
import lightgbm as lgb
dataset_train = dataset[:'2020']

gbm = lgb.LGBMClassifier(n_estimators=100, random_state=5, learning_rate=0.01)

gbm.fit(dataset_train[feature_names], dataset_train['return'] > 1)


LGBMClassifier(learning_rate=0.01, random_state=5)

## 神經網路Preprocessing

In [26]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

dataset_scaled = ss.fit_transform(dataset)
dataset_scaled = pd.DataFrame(dataset_scaled, columns=dataset.columns, index=dataset.index)
dataset_scaled['return'] = dataset['return']
dataset_scaled.describe()

,RSIb,RSIb2,RSIb3,RSIb4,RSIb5,RSIb6,MOMb,KDb,KDb2,KDb3,...,b7,LINEARREG_SLOPE0,LINEARREG_SLOPE1,ADXR0,ADXR1,ADXR2,ADXR3,ADXR4,ADXR5,return
count,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,...,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,7.715000e+04,77150.000000
mean,4.062826e-16,1.479787e-15,-3.589392e-15,-1.782172e-15,-3.785165e-15,-2.730420e-15,2.518904e-17,2.170367e-17,1.182175e-17,-3.735760e-18,...,-3.329057e-15,-1.733646e-16,-5.333675e-17,-2.673227e-16,-3.543561e-16,-3.209962e-16,-7.830130e-17,3.978700e-16,-3.863490e-16,1.000135
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,...,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,0.008257
min,-4.368989e+00,-4.506759e+00,-4.762963e+00,-4.738700e+00,-4.702210e+00,-4.630653e+00,-8.883580e+00,-2.586458e+00,-2.685999e+00,-2.663557e+00,...,-5.480993e+00,-9.322652e+00,-7.097892e+00,-2.105435e+00,-2.011292e+00,-1.696597e+00,-1.612772e+00,-1.755444e+00,-1.801731e+00,0.924507
25%,-6.821675e-01,-6.833395e-01,-6.700367e-01,-6.656202e-01,-6.610806e-01,-6.519681e-01,-4.595441e-01,-6.391872e-01,-6.442800e-01,-6.282222e-01,...,-5.310826e-01,-4.636567e-01,-4.686256e-01,-7.436176e-01,-6.998969e-01,-7.519128e-01,-7.023951e-01,-6.870651e-01,-7.213463e-01,0.996989
50%,9.697027e-02,1.165786e-01,1.156960e-01,1.205676e-01,1.261177e-01,1.343380e-01,9.098792e-02,-1.326469e-02,-3.171791e-02,-7.717300e-03,...,1.203265e-01,7.747991e-02,8.381842e-02,-1.503141e-01,-2.273822e-01,-2.738363e-01,-2.538710e-01,-2.312040e-01,-2.118342e-01,1.000285
75%,7.223377e-01,7.145231e-01,6.887814e-01,6.946299e-01,6.937432e-01,6.916076e-01,5.701869e-01,6.560760e-01,6.728323e-01,6.015924e-01,...,5.829542e-01,5.610455e-01,5.725418e-01,5.911812e-01,4.673996e-01,5.046496e-01,4.980228e-01,5.069936e-01,5.906690e-01,1.003707
max,3.246771e+00,2.815948e+00,3.017904e+00,3.089250e+00,3.105732e+00,3.115908e+00,5.314910e+00,2.726409e+00,2.592381e+00,2.559921e+00,...,4.869922e+00,4.700631e+00,4.095958e+00,4.476193e+00,4.063373e+00,3.820162e+00,3.563766e+00,3.680746e+00,3.523598e+00,1.087726


In [ ]:
import tqdm

n = 3

X = []
y = []
indexes = []
dataset_scaled_x = dataset_scaled[feature_names]

for i in tqdm.tqdm_notebook(range(0, len(dataset_scaled)-n)):
    X.append(dataset_scaled_x.iloc[i:i+n].values)
    y.append(dataset_scaled['return'].iloc[i+n-1])
    indexes.append(dataset_scaled.index[i+n-1])
#dataset_scaled.head()

In [ ]:
indexes[0]

In [7]:
import numpy as np
X = np.array(X)
y = np.array(y)

In [8]:
indexes = np.array(indexes)
indexes

array([Timestamp('2006-06-06 09:15:00'), Timestamp('2006-06-06 09:30:00'),
       Timestamp('2006-06-06 09:45:00'), ...,
       Timestamp('2022-11-15 10:15:00'), Timestamp('2022-11-15 10:30:00'),
       Timestamp('2022-11-15 10:45:00')], dtype=object)

In [9]:
dataset_scaled.index

DatetimeIndex(['2006-06-05 13:30:00', '2006-06-06 09:00:00',
               '2006-06-06 09:15:00', '2006-06-06 09:30:00',
               '2006-06-06 09:45:00', '2006-06-06 10:00:00',
               '2006-06-06 10:15:00', '2006-06-06 10:30:00',
               '2006-06-06 10:45:00', '2006-06-06 11:00:00',
               ...
               '2022-11-14 13:30:00', '2022-11-15 09:00:00',
               '2022-11-15 09:15:00', '2022-11-15 09:30:00',
               '2022-11-15 09:45:00', '2022-11-15 10:00:00',
               '2022-11-15 10:15:00', '2022-11-15 10:30:00',
               '2022-11-15 10:45:00', '2022-11-15 11:00:00'],
              dtype='datetime64[ns]', name='date', length=77150, freq=None)

## 神經網路 Model

In [10]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.models.Sequential()
#model.add(keras.layers.Dense(100, activation="relu", input_shape=(len(feature_names),)))
model.add(layers.LSTM(100, return_sequences=True, input_shape=X[0].shape))
model.add(layers.LSTM(100))
model.add(layers.Dense(8))
model.add(layers.Dense(1,kernel_initializer="uniform",activation='linear'))

adam = keras.optimizers.Adam(0.0006)

model.compile(optimizer=adam, loss="binary_crossentropy", metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 100)            57200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 8)                 808       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 138,417
Trainable params: 138,417
Non-trainable params: 0
_________________________________________________________________


## 神經網路訓練

In [11]:
indexes

array([Timestamp('2006-06-06 09:15:00'), Timestamp('2006-06-06 09:30:00'),
       Timestamp('2006-06-06 09:45:00'), ...,
       Timestamp('2022-11-15 10:15:00'), Timestamp('2022-11-15 10:30:00'),
       Timestamp('2022-11-15 10:45:00')], dtype=object)

In [12]:
# dataset_scaled_train = dataset_scaled[:'2020']

import datetime
X_train = X[indexes < datetime.datetime(2021, 1, 1)]
y_train = y[indexes < datetime.datetime(2021, 1, 1)]

checkpoint_filepath = './checkpoint_u22'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

history = model.fit(
    X_train,
    y_train > 1,
    batch_size=5000,
    epochs=300,
    validation_split=0.2,
    callbacks=[model_checkpoint_callback])

Epoch 1/300
11/11 [==============================] - 4s 349ms/step - loss: 3.6262 - accuracy: 0.4787 - val_loss: 2.8150 - val_accuracy: 0.4543
Epoch 2/300
11/11 [==============================] - 3s 295ms/step - loss: 1.9158 - accuracy: 0.4787 - val_loss: 1.9351 - val_accuracy: 0.4543
Epoch 3/300
11/11 [==============================] - 3s 278ms/step - loss: 1.5633 - accuracy: 0.4787 - val_loss: 1.6003 - val_accuracy: 0.4543
Epoch 4/300
11/11 [==============================] - 3s 282ms/step - loss: 1.3227 - accuracy: 0.4787 - val_loss: 1.3383 - val_accuracy: 0.4543
Epoch 5/300
11/11 [==============================] - 4s 321ms/step - loss: 1.1098 - accuracy: 0.4787 - val_loss: 1.1165 - val_accuracy: 0.4543
Epoch 6/300
11/11 [==============================] - 3s 272ms/step - loss: 0.9303 - accuracy: 0.4819 - val_loss: 0.9410 - val_accuracy: 0.4628
Epoch 7/300
11/11 [==============================] - 5s 431ms/step - loss: 0.8148 - accuracy: 0.4924 - val_loss: 0.8422 - val_accuracy: 0.4647

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

## 回測

In [ ]:
ey = model.predict(X)
ey = pd.Series(ey.swapaxes(0,1)[0], index=indexes)
ey.plot()


In [ ]:
eq = twii[indexes]
returns = (eq.shift(-1) - eq)

signal = (ey > ey.quantile(0.6)).rolling(10).sum() > 0
signal = signal.shift(1).fillna(False)

eq = (returns[signal]['2021':]).cumsum()
eq.plot()

In [ ]:
(signal.astype(int).diff().abs().fillna(0) * 3)['2021':].sum()